In [73]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version


# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [74]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [75]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_sales_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [76]:
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [77]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales_data')



In [78]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql(
    """
    SELECT EXTRACT(YEAR FROM date) AS year,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales_data
    WHERE bedrooms == 4
    GROUP BY year
    ORDER BY year
    """
 ).show()



+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [79]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql(
    """
    SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales_data
    WHERE bedrooms == 3 AND bathrooms == 3
    GROUP BY date_built
    ORDER BY date_built
    """
).show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [80]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
    SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales_data
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""").show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [81]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(price)/AVG(view), 2) AS average_price_per_view
    FROM home_sales_data
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------------------+
|average_price_per_view|
+----------------------+
|              10719.21|
|              10752.44|
|              11639.59|
|              10602.25|
|              11098.16|
|              10995.07|
|              11032.32|
|              10547.85|
|               12498.6|
|              11807.27|
|              44599.26|
|              12447.63|
|              11724.08|
|              12325.12|
|              12447.03|
|              12427.49|
|              13300.39|
|              12457.42|
|              12969.49|
|              13005.84|
+----------------------+
only showing top 20 rows

--- 0.6251518726348877 seconds ---


In [82]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales_data")

DataFrame[]

In [83]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data')

True

In [84]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(price)/AVG(view), 2) AS average_price_per_view
    FROM home_sales_data
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------------------+
|average_price_per_view|
+----------------------+
|              10719.21|
|              10752.44|
|              11639.59|
|              10602.25|
|              11098.16|
|              10995.07|
|              11032.32|
|              10547.85|
|               12498.6|
|              11807.27|
|              44599.26|
|              12447.63|
|              11724.08|
|              12325.12|
|              12447.03|
|              12427.49|
|              13300.39|
|              12457.42|
|              12969.49|
|              13005.84|
+----------------------+
only showing top 20 rows

--- 0.4512341022491455 seconds ---


In [85]:
# Run the uncached query and store the execution time
start_time_uncached = time.time()
spark.sql("""
    SELECT ROUND(AVG(price)/AVG(view), 2) AS average_price_per_view
    FROM home_sales_data
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""").show()
uncached_runtime = time.time() - start_time_uncached

# Run the cached query and store the execution time
start_time_cached = time.time()
spark.sql("""
    SELECT ROUND(AVG(price)/AVG(view), 2) AS average_price_per_view
    FROM home_sales_data
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""").show()
cached_runtime = time.time() - start_time_cached

# Print the runtimes and the difference
print("Cached runtime:", cached_runtime)
print("Uncached runtime:", uncached_runtime)
print("Difference:", uncached_runtime - cached_runtime)

+----------------------+
|average_price_per_view|
+----------------------+
|              10719.21|
|              10752.44|
|              11639.59|
|              10602.25|
|              11098.16|
|              10995.07|
|              11032.32|
|              10547.85|
|               12498.6|
|              11807.27|
|              44599.26|
|              12447.63|
|              11724.08|
|              12325.12|
|              12447.03|
|              12427.49|
|              13300.39|
|              12457.42|
|              12969.49|
|              13005.84|
+----------------------+
only showing top 20 rows

+----------------------+
|average_price_per_view|
+----------------------+
|              10719.21|
|              10752.44|
|              11639.59|
|              10602.25|
|              11098.16|
|              10995.07|
|              11032.32|
|              10547.85|
|               12498.6|
|              11807.27|
|              44599.26|
|              12447.63|

In [86]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")


In [87]:
# 11. Read the parquet formatted data.
parquet_home_df = spark.read.parquet("home_sales_partitioned")

In [88]:
# 12. Create a temporary table for the parquet data.
parquet_home_df.createOrReplaceTempView('parquet_temp_home')

In [89]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(price)/AVG(view), 2) AS average_price_per_view
    FROM parquet_temp_home
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------------------+
|average_price_per_view|
+----------------------+
|              10719.21|
|              10752.44|
|              11639.59|
|              10602.25|
|              11098.16|
|              10995.07|
|              11032.32|
|              10547.85|
|               12498.6|
|              11807.27|
|              44599.26|
|              12447.63|
|              11724.08|
|              12325.12|
|              12447.03|
|              12427.49|
|              13300.39|
|              12457.42|
|              12969.49|
|              13005.84|
+----------------------+
only showing top 20 rows

--- 0.6484873294830322 seconds ---


In [91]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales_data")

DataFrame[]

In [92]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_data')


False